In [1]:
import pandas as pd
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from nbimporter import NotebookLoader

warnings.filterwarnings("ignore")

In [2]:
ma = pd.read_csv("../16 Mycoplosma/ma.csv")
map = pd.read_csv("../16 Mycoplosma/map.csv")
marth = pd.read_csv("../16 Mycoplosma/marth.csv")
mcap = pd.read_csv("../16 Mycoplosma/mcap.csv")
mcon = pd.read_csv("../16 Mycoplosma/mcon.csv")
mcro = pd.read_csv("../16 Mycoplosma/mcro.csv")
mgal = pd.read_csv("../16 Mycoplosma/mgal.csv")
mhom = pd.read_csv("../16 Mycoplosma/mhom.csv")
mhyoJ = pd.read_csv("../16 Mycoplosma/mhyoJ.csv")
mhyo232 = pd.read_csv("../16 Mycoplosma/mhyo232.csv")
mhyo7448 = pd.read_csv("../16 Mycoplosma/mhyo7448.csv")
mm163K = pd.read_csv("../16 Mycoplosma/mm163K.csv")
mms = pd.read_csv("../16 Mycoplosma/mms.csv")
mpHF = pd.read_csv("../16 Mycoplosma/mpHF.csv")
mpM = pd.read_csv("../16 Mycoplosma/mpM.csv")
ms53 = pd.read_csv("../16 Mycoplosma/ms53.csv")

In [3]:
def generate_train_test_dataset(index):
    dataset_names = [ma, map, marth, mcap, mcon, mcro, mgal, mhom, mhyoJ, mhyo232, mhyo7448, mm163K, mms, mpHF, mpM, ms53]
    merged_train = dataset_names[:index]+dataset_names[index+1:]
    train_data = pd.concat(merged_train)
    test_data = dataset_names[index]
    return train_data,test_data

In [4]:
def handle_dataset(dataset):
    columns_to_drop = ['identity','name', 'Amino acid', 'protein_sequence', 'essentiality']
    Xs = dataset.drop(columns = columns_to_drop)
    y = dataset['essentiality']
    return Xs,y

In [5]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [6]:
from sklearn.linear_model import LassoCV

lassoCV = LassoCV(cv=20,random_state=10)
all_feature_names = ["GC_Content","CAI","A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V","nSE2","nSE3","nGE2","nGE3"]

In [7]:
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 100),  # 树的数量
    'max_depth': hp.quniform('max_depth', 3, 10, 1),  # 每棵树的最大深度
    'learning_rate': hp.loguniform('learning_rate', -3, 0),  # 学习率
    'subsample': hp.uniform('subsample', 0.5, 1.0),  # 每棵树的样本采样比例
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),  # 每棵树的特征采样比例
    'gamma': hp.loguniform('gamma', -3, 2),  # 节点分裂时损失函数减小值的最小值要求
    'reg_alpha': hp.loguniform('reg_alpha', -3, 2),  # L1 正则化项系数
    'reg_lambda': hp.loguniform('reg_lambda', -3, 2),  # L2 正则化项系数
}

In [8]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


def xgb_ac_cv(params):
    scorer = make_scorer(accuracy_score)
    model = XGBClassifier(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        objective='binary:logistic'  # 二分类问题
    )
    scorer = make_scorer(accuracy_score)
    score = -cross_val_score(model, X_resampled, y_resampled, cv=5,scoring=scorer).mean()
    return score

In [9]:
scores = []
score = 0
models = []
for j in range(0,3):
    for i in range(0,15):
        train_data, test_data = generate_train_test_dataset(i)
        Xs_train,y_train = handle_dataset(train_data)
        Xs_test,y_test = handle_dataset(test_data)
        rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
        X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
        lassoCV.fit(X_resampled,y_resampled)
        selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
        trials = Trials()
        best=fmin(fn=xgb_ac_cv, # function to optimize
              space=space, 
              algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
              max_evals=50, # maximum number of iterations
              trials=trials, # logging
              rstate=np.random.default_rng(42) # fixing random state for the reproducibility
        )
        model = XGBClassifier(
        n_estimators=int(best['n_estimators']),
        max_depth=int(best['max_depth']),
        learning_rate=best['learning_rate'],
        subsample=best['subsample'],
        colsample_bytree=best['colsample_bytree'],
        gamma=best['gamma'],
        reg_alpha=best['reg_alpha'],
        reg_lambda=best['reg_lambda'],
        objective='binary:logistic'  # 二分类问题
    )
        models.append(model)
        model.fit(X_resampled[selected_features],y_resampled)
        tpe_test_score=accuracy_score(y_test, model.predict(Xs_test[selected_features]))
        scores.append(tpe_test_score)
        print(tpe_test_score)
        if score < tpe_test_score:
            score = tpe_test_score
            best_model = model

100%|██████████| 50/50 [01:04<00:00,  1.30s/trial, best loss: -0.8669388954171563]
0.8673740053050398
100%|██████████| 50/50 [01:17<00:00,  1.54s/trial, best loss: -0.8780219780219781]
0.8136645962732919
100%|██████████| 50/50 [01:09<00:00,  1.38s/trial, best loss: -0.8689498236431904]
0.6540880503144654
100%|██████████| 50/50 [01:31<00:00,  1.83s/trial, best loss: -0.858260776667913] 
0.9142857142857143
100%|██████████| 50/50 [01:30<00:00,  1.81s/trial, best loss: -0.8659421539876948]
0.8803680981595092
100%|██████████| 50/50 [01:28<00:00,  1.78s/trial, best loss: -0.8780886196246138]
0.8898305084745762
100%|██████████| 50/50 [01:06<00:00,  1.33s/trial, best loss: -0.8723560517038778]
0.7191283292978208
100%|██████████| 50/50 [01:23<00:00,  1.66s/trial, best loss: -0.8745788051290898]
0.9387096774193548
100%|██████████| 50/50 [01:24<00:00,  1.69s/trial, best loss: -0.8838605074524974]
0.6878612716763006
100%|██████████| 50/50 [01:20<00:00,  1.62s/trial, best loss: -0.8623884402889928]

In [10]:
print(scores)

[0.8673740053050398, 0.8136645962732919, 0.6540880503144654, 0.9142857142857143, 0.8803680981595092, 0.8898305084745762, 0.7191283292978208, 0.9387096774193548, 0.6878612716763006, 0.8548621190130624, 0.8988439306358381, 0.931129476584022, 0.9002493765586035, 0.89, 0.45951417004048584, 0.8673740053050398, 0.8136645962732919, 0.6540880503144654, 0.9142857142857143, 0.8803680981595092, 0.8898305084745762, 0.7191283292978208, 0.9387096774193548, 0.6878612716763006, 0.8548621190130624, 0.8988439306358381, 0.931129476584022, 0.9002493765586035, 0.89, 0.45951417004048584, 0.8673740053050398, 0.8136645962732919, 0.6540880503144654, 0.9142857142857143, 0.8803680981595092, 0.8898305084745762, 0.7191283292978208, 0.9387096774193548, 0.6878612716763006, 0.8548621190130624, 0.8988439306358381, 0.931129476584022, 0.9002493765586035, 0.89, 0.45951417004048584]


In [10]:
from joblib import dump

dump(best_model, '../../model/XGB_model_lasso.joblib')

['../../model/XGB_model_lasso.joblib']

In [11]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5950068203970165, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.20261344066604717,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08407518937092857,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=700, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [12]:
train_data, test_data = generate_train_test_dataset(12)
Xs_train,y_train = handle_dataset(train_data)
Xs_test,y_test = handle_dataset(test_data)
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = rus.fit_resample(Xs_train, y_train)
lassoCV.fit(X_resampled,y_resampled)
selected_features = [feature for feature, coef in zip(all_feature_names, lassoCV.coef_) if coef !=0]
print(selected_features)

['GC_Content', 'CAI', 'A', 'N', 'D', 'H', 'M', 'nSE3', 'nGE3']
